In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [16]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge

In [17]:
from sklearn.metrics import mean_squared_error

In [18]:
from sklearn.preprocessing import PolynomialFeatures

In [19]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import geopandas as gpd
import shapely

In [21]:
neighbor = gpd.GeoDataFrame.from_file('../geo/neighborhoods.geojson')
comm = pd.read_csv('../geo/zip_comm.csv')
census = pd.read_csv('../geo/census_zip.csv')

In [22]:
comm['ZCTA5'] = comm['ZCTA5'].astype('int').astype('str')
comm['CHGOCA'] = comm['CHGOCA'].astype('int').astype('str')

In [23]:
census['zip'] = census['zip'].astype('int').astype('str')
census['tract'] = census['tract'].astype('str')

In [24]:
data = ['EMSOD','HCSBDP','HCSBHP','HCSNVP',
             'HCSSPDP','HDBHR','HDDRR','HDSZR',
        'MEODR','YRDEP','VRALR','VRDIDR', 'VRDOR', 'VRSUR']

In [25]:
files = [f'../data/{file}.csv' for file in data]

In [26]:
mh = pd.concat(map(pd.read_csv, files),axis = 0)
mh = mh[mh['p'].isnull()]
mh = mh[mh['l'] != 'place']
mh['g'] = mh['g'].astype('str')

mh = mh.merge(comm, how = 'left', left_on = 'g', right_on = 'ZCTA5')

In [27]:
mh.loc[mh['g'].str.len() == 5,'g'] = mh['CHGOCA']
mh['g'] = mh['g'].str.replace('1714000-','')

In [28]:
neighbor = neighbor.merge(mh, how = 'left', left_on = 'area_numbe', right_on = 'g')

In [29]:
neighbor['long'] = neighbor['geometry'].centroid.x
neighbor['lat'] = neighbor['geometry'].centroid.y

C:\Users\Finley & Stein\AppData\Local\Temp\ipykernel_4492\498829464.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighbor['long'] = neighbor['geometry'].centroid.x
C:\Users\Finley & Stein\AppData\Local\Temp\ipykernel_4492\498829464.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  neighbor['lat'] = neighbor['geometry'].centroid.y


----

##  CZA

In [30]:
CZA = pd.read_csv(f'../data/CZA.csv')
CZB = pd.read_csv(f'../data/CZB.csv')
CZD = pd.read_csv(f'../data/CZD.csv')
CZH = pd.read_csv(f'../data/CZH.csv')
CZL = pd.read_csv(f'../data/CZL.csv')
CZM = pd.read_csv(f'../data/CZM.csv')
CZO = pd.read_csv(f'../data/CZO.csv')
CZP = pd.read_csv(f'../data/CZP.csv')
CZR = pd.read_csv(f'../data/CZR.csv')
CZS = pd.read_csv(f'../data/CZS.csv')
CZT = pd.read_csv(f'../data/CZT.csv')
CZU = pd.read_csv(f'../data/CZU.csv')
CZV = pd.read_csv(f'../data/CZV.csv')

In [31]:
df = CZA

In [32]:
def transform(df):
    df.drop(df[df['p'].notnull()].index, inplace = True)
    df.drop(df[df['l'] == 'place'].index, inplace = True)

    df = df.merge(census, how = 'left', left_on = 'g', right_on = 'tract')
    df['zip'] = df['zip'].astype('str')
    df.loc[df['l'] == 'tract-2010','g'] = df['zip']

    df = df.merge(comm, how = 'left', left_on = 'g', right_on = 'ZCTA5')
    df['zip'] = df['zip'].astype('str')
    df.loc[df['l'] == 'tract-2010','g'] = df['zip']

    df.loc[df['g'].str.len() == 5,'g'] = df['CHGOCA']
    df['g'] = df['g'].str.replace('1714000-','')

    df = neighbor.merge(df, how = 'left', left_on = ['g','d'], right_on = ['g','d'])
    df.dropna(subset = 'v_y',inplace = True)
    df.rename(columns = {'a_x':'mh_stat','v_x':'mh_val','v_y':'crim_val'}, inplace = True)
    return df

In [ ]:
values = []
for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    poly = PolynomialFeatures()
    X_train = poly.fit_transform(X_train)
    X_test = poly.transform(X_test)
    
    base = [np.mean(y_train)] * len(y_test)
    baseline = mean_squared_error(y_test, base, squared=False)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr_train = mean_squared_error(y_train, lr.predict(X_train), squared=False)
    lr_test = mean_squared_error(y_test, lr.predict(X_test), squared=False)
    print(('lr',lr_train,lr_test))
    
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    lasso_train = mean_squared_error(y_train,lasso.predict(X_train), squared = False)
    lasso_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)
    print(('lasso',lasso_train, lasso_test))
    
    rg = Ridge()
    rg.fit(X_train, y_train)
    rg_train = mean_squared_error(y_train,rg.predict(X_train), squared = False) 
    rg_test = mean_squared_error(y_test, rg.predict(X_test), squared=False)
    print(('rg',rg_train,rg_test))
    
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    knn_train = mean_squared_error(y_train,knn.predict(X_train), squared = False)
    knn_test = mean_squared_error(y_test, knn.predict(X_test), squared=False)
    print(('knn',knn_train,knn_test))

    dt = DecisionTreeRegressor()
    dt.fit(X_train, y_train)
    dt_train = mean_squared_error(y_train,dt.predict(X_train), squared = False) 
    dt_test = mean_squared_error(y_test, dt.predict(X_test), squared=False)
    print(('dt',dt_train,dt_test))

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    rf_train = mean_squared_error(y_train,rf.predict(X_train), squared = False)
    rf_test = mean_squared_error(y_test, rf.predict(X_test), squared=False)
    print(('rf',rf_train,rf_test))

    abr = AdaBoostRegressor()
    abr.fit(X_train, y_train)
    abr_train = mean_squared_error(y_train,abr.predict(X_train), squared = False)
    abr_test = mean_squared_error(y_test, abr.predict(X_test), squared=False)
    print(('abr',abr_train,abr_test))
    values.append([baseline, lr_train, lr_test, lasso_train, lasso_test, rg_train, rg_test, knn_train, knn_test, dt_train, dt_test, rf_train, rf_test, abr_train, abr_test])

('lr', 10.595690246612586, 10.503987616091067)
('lasso', 11.805790967379133, 11.764964385227655)
('rg', 10.532918472290946, 10.451252699398585)


In [ ]:
errors = pd.DataFrame(values, columns = ['baseline', 'lr_train', 'lr_test', 'lasso_train', 'lasso_test', 
                                'rg_train', 'rg_test', 'knn_train', 'knn_test', 
                                'dt_train', 'dt_test', 'rf_train', 'rf_test', 
                                'abr_train', 'abr_test'])

In [ ]:
columns = pd.DataFrame(['CZA','CZB', 'CZD', 'CZH', 'CZL','CZM', 'CZO', 'CZP', 'CZR', 'CZS', 'CZT', 'CZV'])

In [ ]:
errors = pd.concat([errors,columns], axis = 1)

In [ ]:
errors.set_index(0, inplace = True)

In [ ]:
errors.to_csv('../data/errors.csv')

In [ ]:
errors.describe()

In [ ]:
for stat in [CZA, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    poly = PolynomialFeatures()
    X_train = poly.fit_transform(X_train)
    X_test = poly.transform(X_test)
    
    gs = GridSearchCV(KNeighborsRegressor(),
                      param_grid = {'n_neighbors':[3,5,7,10,15],
                                   'p':[1,2]},
                      cv = 5,
                      n_jobs = -1)
    
    gs.fit(X_train, y_train)
    print(gs.best_params_)

{'n_neighbors': 3, 'p': 1}


In [ ]:
, CZB, CZD, CZH, CZL, CZM, CZO, CZP, CZR, CZS, CZT, CZV

In [221]:
for stat in [CZA]:
    X = transform(stat)[['mh_stat','mh_val','lat','long','d','crim_val']]
    avgs = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].agg('mean')
    X['crim_val'] = X.groupby(['mh_stat','mh_val','lat','long'])['crim_val'].transform(lambda x: avgs[x.name])
    
    y = X['crim_val']
    X = X.drop(columns = ['crim_val','d'])
    
    X = pd.get_dummies(X, columns = ['mh_stat'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)
    
    poly = PolynomialFeatures()
    X_train = poly.fit_transform(X_train)
    X_test = poly.transform(X_test)
    
    knn = KNeighborsRegressor(n_neighbors=3)
    knn.fit(X_train,y_train)
    y_pred = knn.predict(X_test)

In [223]:
[X_test,y_pred]

[array([[ 1.        , -0.72260815, -1.51503973, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.091508  , -0.48239101, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.4717863 ,  1.03157575, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        ...,
        [ 1.        ,  2.837444  ,  0.4378657 , ...,  3.01478808,
         -1.0050396 ,  0.33504995],
        [ 1.        , -0.3544664 ,  1.23083567, ...,  0.33169827,
          0.3333699 ,  0.33504995],
        [ 1.        , -0.86824665, -0.27181914, ...,  0.33169827,
         -0.99498567,  2.98462961]]),
 array([50.63247933, 18.938757  ,  7.79390601, ..., 63.73305938,
        13.3781609 , 23.45584757])]